# Test połączenia + importy

In [1]:
import mysql.connector
from faker import Faker
import random
from datetime import datetime, timedelta
import math
import unicodedata

DB_CONFIG = {
    'host': 'giniewicz.it',
    'user': 'team17',
    'password': 'te@mlig',
    'database': 'team17',
    'autocommit': False
}

con = mysql.connector.connect(**DB_CONFIG)

if(con):
    print("Połączenie udane")
else:
    print("Połączenie nieudane")

con.close()

Połączenie udane


# UZUPEŁNIANIE TABELI PO KOLEI

## Stacje

In [2]:
DISTANCES = {
    'Ziemia':  {'r': 6371,        'opis': 'Stacja przy powierzchni Ziemi'},
    'Ksiezyc': {'r': 384_400,     'opis': 'Orbitalna stacja Księżyca'},
    'Mars':    {'r': 228_000_000, 'opis': 'Orbitalna stacja Marsa'},
    'Merkury': {'r': 58_000_000,  'opis': 'Orbitalna stacja Merkurego'},
    'Wenus':   {'r': 108_000_000, 'opis': 'Orbitalna stacja Wenus'},
    'Jowisz':  {'r': 778_000_000, 'opis': 'Orbitalna stacja Jowisza'},
    'Europa':  {'r': 778_670_900, 'opis': 'Orbitalna stacja Europy'}
}

def fill_STACJE():
    conn = mysql.connector.connect(**DB_CONFIG)
    cur = conn.cursor()
    try:
        cur.execute('SET FOREIGN_KEY_CHECKS=0;')
        cur.execute('TRUNCATE TABLE STACJE;')

        entries = []
        bodies = list(DISTANCES.items())
        n = len(bodies)
        theta = math.pi / 2
        for idx, (name, data) in enumerate(bodies, start=1):
            base_r = data['r']
            opis = data['opis']
            phi = 2 * math.pi * (idx - 1) / n
            r = base_r + 100  # 100 km nad powierzchnią/orbitą
            entries.append((idx, r, theta, phi, opis))

        cur.executemany(
            'INSERT INTO STACJE (ID_STACJI, POLOZENIE_R, POLOZENIE_THETA, POLOZENIE_PHI, OPIS_STACJI) '
            'VALUES (%s, %s, %s, %s, %s)',
            entries
        )

        cur.execute('SET FOREIGN_KEY_CHECKS=1;')
        conn.commit()
        print('STACJE zostały wypełnione wraz z opisami.')
    except Exception:
        conn.rollback()
        raise
    finally:
        cur.close()
        conn.close()

if __name__ == '__main__':
    fill_STACJE()

STACJE zostały wypełnione wraz z opisami.


## Osoby i bliscy

In [ ]:
MIN_PARENT_AGE_DIFF = 15
NUM_PEOPLE = 1000

STATION_TO_CITIES = {
    2: ["Selena", "Baza Księżycowa Alfa", "Kopernik Domek", "Tycho Główne", "Morze Spokoju", "Nowy Armstrong", "Selenopol", "Plato Stacja"],
    3: ["Ares Główny", "Góra Olimpu", "Dolina Marinerisa", "Cydonia Wzgórza", "Marsport", "Phobos Kolonia", "Terra Nova", "Elonowo"]
}

STATION_TO_STREETS = {
    2: ["Aleja Apollo", "Aleja Armstronga", "Selenia Obok", "Orbita Księżyca", "Grzbiet Krateru", "Droga Helu-3", "Droga Dostępu do Kopuły", "Ścieżka Wschodu Księżyca", "Wzgórza Niebieskie", "Silicon Valley Extensja"],
    3: ["Ulica Ciekawości", "Ścieżka Perseverance", "Kanion Marinerisa", "Droga łazika", "Bulwar Czerwonej Planety", "Ścieżka Pyłu Marsjańskiego", "Avenue Terraformacji", "Widok na Phobosa", "Grzbiet Oxonii", "Autostrada Elona"]
}

def normalize_text(text):
    """Usuwa znaki diakrytyczne i konwertuje tekst na małe litery"""
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    ).lower()

def generate_email(first_name, last_name):
    """Generuje e-mail na podstawie imienia i nazwiska"""
    first = normalize_text(first_name)
    last = normalize_text(last_name)
    
    # Wzorce e-maili
    patterns = [
        f"{first}.{last}",                                 # jan.kowalski
        f"{first[0]}{last}",                               # jkowalski
        f"{first}_{last}{random.randint(1, 999)}",         # jan_kowalski123
        f"{first}{last}",                                  # jakowalski
        f"{first}.{last[0]}",                              # jan.k
        f"{first}{last[:3]}",                              # jakowa
        f"{first[0]}.{last}{random.randint(1, 999)}",      # j.kowalski123
        f"{first}{random.randint(1, 999)}"                 # jan123
    ]
    
    domains = ["gmail.com", "yahoo.com", "hotmail.com", "outlook.com", "example.com", "interia.pl"]
    
    return f"{random.choice(patterns)}@{random.choice(domains)}"

def fill_OSOBY_BLISCY():
    fake = Faker('pl_PL')
    conn = mysql.connector.connect(**DB_CONFIG)
    cur  = conn.cursor()

    try:
        cur.execute("SET FOREIGN_KEY_CHECKS=0;")
        cur.execute("TRUNCATE TABLE BLISCY;")
        cur.execute("TRUNCATE TABLE OSOBY;")
        cur.execute("SET FOREIGN_KEY_CHECKS=1;")
        conn.commit()

        persons = []
        for pid in range(1, NUM_PEOPLE+1):
            dob = fake.date_of_birth(minimum_age=18, maximum_age=80)
            stid = random.choices([1,2,3], [0.5, 0.2, 0.3], k=1)[0]

            if stid == 1:
                city = fake.city()
                street = fake.street_address()
            else:
                city = random.choice(STATION_TO_CITIES[stid])
                street = random.choice(STATION_TO_STREETS[stid])
            
            first_name = fake.first_name()
            last_name = fake.last_name()
              
            persons.append((
                pid,
                first_name, last_name, dob,
                None, fake.phone_number(),
                street, city, fake.postcode(), stid
            ))

        contacts_set = set()
        contacts = []
        partners_id = [0] * (NUM_PEOPLE+1) # tablica do sprawdzania czy ktoś jest już partnerem

        for pid, first, last, dob, email, phone, street, city, postal, stid in persons:
            for _ in range(random.randint(1, 3)):
                rel = random.choice(['rodzic', 'partner', 'przyjaciel'])
                if rel == 'rodzic':
                    candidates = [p for p in persons if p[0] != pid and p[3] < dob - timedelta(days=365*MIN_PARENT_AGE_DIFF)] # pid jest rodzicem bid
                    if not candidates:
                        rel = 'przyjaciel' #dodanie obsługi ludzi za młodych na dzieci
                if rel == 'partner':
                    candidates = [p for p in persons if p[0] != pid and abs((p[3] - dob).days) < 365*10]
                else:
                    candidates = [p for p in persons if p[0] != pid and abs((p[3] - dob).days) < 365*30]

                if not candidates:
                    continue

                bid = random.choice(candidates)[0]
                if bid != pid and (pid, bid) not in contacts_set and (bid, pid) not in contacts_set: 
                    if rel == 'partner':
                        is_relation_valid = True #zabezpieczenie przed więcej niż jedną relacją partnera
                        for element in contacts:
                            if (element[0] == pid or element[1] == pid) and element[2] == 'partner':
                                is_relation_valid = False
                                break
                            if (element[0] == bid or element[1] == bid) and element[2] == 'partner':
                                is_relation_valid = False
                                break
                            
                        if is_relation_valid:
                            contacts.append((pid, bid, rel))
                            contacts_set.add((pid, bid))
                            personal_data_p = persons[pid - 1]
                            personal_data_b = persons[bid - 1]
                            persons[bid - 1] = ( #zmiana nazwiska partnera
                                personal_data_b[0], personal_data_b[1], personal_data_p[2],
                                personal_data_b[3], personal_data_b[4], personal_data_b[5],
                                personal_data_b[6], personal_data_b[7], personal_data_b[8],
                                personal_data_b[9] 
                            )
                            partners_id[pid] = bid #zapisanie partnera w tablicy dla przyśpieszania wyszukiwania
                            partners_id[bid] = pid 
                            
                        else: #friend zone xd żeby każdy miał przynajmniej jedną bliską osobę
                            contacts.append((pid, bid, 'przyjaciel'))
                            contacts_set.add((pid, bid))
                            
                            
                    elif rel == 'przyjaciel':
                        contacts.append((pid, bid, rel))
                        contacts_set.add((pid, bid))
                    elif rel == 'rodzic':
                        is_child = False
                        for element in contacts:
                            if element[1] == bid and element[2] == 'rodzic': #bid ma rodzica czyli jest już czyimś dzieckiem - potencjalny drugi rodzic potem
                                is_child = True
                                break
                        if not is_child:
                            contacts.append((pid, bid, 'rodzic')) #pid jest rodzicem bid
                            contacts_set.add((pid, bid))
                        else: #zostają przyjaciółmi żeby każdy miał przynajmniej jedną bliską osobę
                            contacts.append((pid, bid, 'przyjaciel'))
                            contacts_set.add((pid, bid))
                            
        #nazwiska i drugi rodzic
        new_parents = []
        for contact in contacts:
            if contact[2] == 'rodzic':
                personal_data_p = persons[contact[0] - 1]
                personal_data_b = persons[contact[1] - 1]
                persons[contact[1] - 1] = ( 
                    personal_data_b[0], personal_data_b[1], personal_data_p[2],
                    personal_data_b[3], personal_data_b[4], personal_data_b[5],
                    personal_data_b[6], personal_data_b[7], personal_data_b[8],
                    personal_data_b[9] 
                )
            partner_id = partners_id[contact[0]]
            if partner_id != 0:  # Dodaj tylko jeśli partner_id jest poprawnym ID
                if (partner_id, contact[1]) not in contacts_set and (contact[1], partner_id) not in contacts_set:
                    new_parents.append((partner_id, contact[1], 'rodzic'))
                    contacts_set.add((partner_id, contact[1]))

        contacts.extend(new_parents) #dodanie drugiego rodzica do kontaktów

        contacts = [c for c in contacts if c[0] != c[1]] # usuń relacje z samym sobą

        # usuwanie duplikatów relacji
        seen = set()
        unique_contacts = []
        for c in contacts:
            key = (c[0], c[1])
            if key not in seen:
                seen.add(key)
                unique_contacts.append(c)
        contacts = unique_contacts

        used_emails = set()

        for i in range(len(persons)):
            pid, first_name, last_name, dob, _, phone, street, city, postal, stid = persons[i]
            
            email = generate_email(first_name, last_name)
            
            original_email = email
            counter = 1
            while email in used_emails:
                local_part, domain = original_email.rsplit('@', 1)
                email = f"{local_part}_{counter}@{domain}"
                counter += 1
                if counter > 1000:
                    raise ValueError("Za dużo duplikatów e-maili. Popraw logikę generacji.")
            used_emails.add(email)
            
            persons[i] = (
                pid, first_name, last_name, dob,
                email, phone, street, city, postal, stid
            )

        cur.executemany(
            """
            INSERT INTO OSOBY
              (ID_OSOBY, IMIE, NAZWISKO, DATA_URODZENIA, EMAIL,
               TELEFON, ULICA, MIEJSCOWOSC, KOD_POCZTOWY, ID_STACJI)
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
            """,
            persons
        )
        conn.commit()

        if contacts:
            cur.executemany(
                """
                INSERT INTO BLISCY
                  (ID_OSOBY, ID_BLISKIEGO, TYP_RELACJI)
                VALUES (%s,%s,%s)
                """,
                contacts
            )
            conn.commit()

        print("Tabele OSOBY i BLISCY zostały wypełnione.")

    except Exception:
        conn.rollback()
        raise

    finally:
        cur.close()
        conn.close()

if __name__ == '__main__':
    fill_OSOBY_BLISCY()

Tabele OSOBY i BLISCY zostały wypełnione.


## Pojazdy

In [4]:
PRODUCERS = [
    'SpaceX', 'Blue Origin', 'Rotary Rocket', 'Boeing', 'Orbital ATK', 'Reaction Engines Limited'
]

MODELS = [
    'Falcon 9', 'Antares', 'Atlas V', 'Super Heavy'
]

TYPES = [
    'rakieta zalogowa'
]

NUM_VEHICLES = 10

def fill_POJAZDY():
    conn = mysql.connector.connect(**DB_CONFIG)
    cur = conn.cursor()
    try:
        cur.execute("SET FOREIGN_KEY_CHECKS=0;")
        cur.execute("TRUNCATE TABLE POJAZDY;")
        cur.execute("SET FOREIGN_KEY_CHECKS=1;")
        
        entries = []
        for vid in range(1, NUM_VEHICLES + 1):
            producer = random.choice(PRODUCERS)
            model    = random.choice(MODELS)
            type     = random.choice(TYPES)
            range_km = random.randint(100_000, 1_000_000)   # zasięg [km]
            speed    = random.randint(100_00, 500_00)      # prędkość max [km/h]
            capacity = random.randint(5, 20)             # pojemność [osób]
            entries.append((vid, producer, model, range_km, speed, type, capacity))
        
        cur.executemany(
            """
            INSERT INTO POJAZDY
              (ID_POJAZDU, PRODUCENT, MODEL, ZASIEG_KM, PREDKOSC_MAX, TYP_POJAZDU, POJEMNOSC)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            """,
            entries
        )
        conn.commit()
        print(f"Wstawiono {NUM_VEHICLES} rekordów do POJAZDY.")
    finally:
        cur.close()
        conn.close()

if __name__ == '__main__':
    fill_POJAZDY()

Wstawiono 10 rekordów do POJAZDY.


## Trasy

In [5]:
NUM_ROUTES = 18

START_STATION_WEIGHTS = [0.5, 0.2, 0.3]

def sph_to_cart(r, theta, phi):
    x = r * math.sin(theta) * math.cos(phi)
    y = r * math.sin(theta) * math.sin(phi)
    z = r * math.cos(theta)
    return x, y, z


def fill_TRASY():
    conn = mysql.connector.connect(**DB_CONFIG)
    cur = conn.cursor()

    try:
        cur.execute('SET FOREIGN_KEY_CHECKS=0;')
        cur.execute('TRUNCATE TABLE TRASY;')
        cur.execute('SET FOREIGN_KEY_CHECKS=1;')

        cur.execute('SELECT ID_STACJI, POLOZENIE_R, POLOZENIE_THETA, POLOZENIE_PHI FROM STACJE WHERE ID_STACJI IN (1, 2, 3);')
        start_stations = cur.fetchall()
        start_ids = [sid for sid, _, _, _ in start_stations]
        start_coords = {sid: sph_to_cart(r, theta, phi) for sid, r, theta, phi in start_stations}

        cur.execute('SELECT ID_STACJI, POLOZENIE_R, POLOZENIE_THETA, POLOZENIE_PHI FROM STACJE;')
        all_stations = cur.fetchall()
        all_coords = {sid: sph_to_cart(r, theta, phi) for sid, r, theta, phi in all_stations}
        all_ids = list(all_coords.keys())

        routes = []
        used_pairs = set()
        rid = 1

        possible_pairs = [(a, b) for a in start_ids for b in all_ids if a != b]
        random.shuffle(possible_pairs)

        for a, b in possible_pairs:
            if len(routes) >= NUM_ROUTES:
                break
            x1, y1, z1 = start_coords[a]
            x2, y2, z2 = all_coords[b]
            one_way_dist = math.dist((x1, y1, z1), (x2, y2, z2))
            round_trip_dist = int(round(one_way_dist * 2))
            routes.append((rid, a, b, round_trip_dist))
            rid += 1

        cur.executemany(
            'INSERT INTO TRASY (ID_TRASY, ID_STACJA_START, ID_STACJA_KONIEC, DLUGOSC_TRASY) VALUES (%s, %s, %s, %s)',
            routes
        )
        conn.commit()
        print(f"Wstawiono {len(routes)} tras do TRASY.")

    except Exception as e:
        conn.rollback()
        raise e

    finally:
        cur.close()
        conn.close()

if __name__ == '__main__':
    fill_TRASY()


Wstawiono 18 tras do TRASY.


## Pracownicy

In [ ]:
years_active = 5
total_current = 60
total_past    = 20
positions = ['Pilot', 'Mechanik', 'Specjalista ds. utrzymania czystosci', 'Obsluga lotu']
min_hire_age = 20

def fill_PRACOWNICY():
    conn = mysql.connector.connect(**DB_CONFIG)
    cur  = conn.cursor()

    try:
        cur.execute("SET FOREIGN_KEY_CHECKS=0;")
        cur.execute("TRUNCATE TABLE PRACOWNICY;")
        cur.execute("SET FOREIGN_KEY_CHECKS=1;")
        conn.commit()

        cur.execute("SELECT ID_OSOBY, DATA_URODZENIA FROM OSOBY;")
        persons = cur.fetchall()

        today = datetime.now().date()
        inception = today - timedelta(days=365 * years_active)

        entries = []
        eid = 1

        for pos in positions:
            cutoff = inception - timedelta(days=365 * min_hire_age)
            elig = [pid for pid, dob in persons if dob <= cutoff]
            if not elig:
                continue
            pid = random.choice(elig)
            entries.append((eid, pid, pos, inception, None))
            eid += 1

        remaining_current = total_current - len(positions)
        remaining_total   = remaining_current + total_past
        weights = [i+1 for i in range(years_active)]
        sum_w = sum(weights)
        hires_per_year = [int(remaining_total * w / sum_w) for w in weights]
        diff = remaining_total - sum(hires_per_year)
        for i in range(diff):
            hires_per_year[-1 - (i % years_active)] += 1

        start_dates = []
        for year_idx, count in enumerate(hires_per_year):
            ys = inception + timedelta(days=365 * year_idx)
            ye = inception + timedelta(days=365 * (year_idx+1))
            for _ in range(count):
                delta = (ye - ys).days
                start_dates.append(ys + timedelta(days=random.randint(0, delta)))
        random.shuffle(start_dates)

        for sd in start_dates[:remaining_current]:
            cutoff = sd - timedelta(days=365 * min_hire_age)
            elig = [pid for pid, dob in persons if dob <= cutoff]
            if not elig:
                continue
            pid = random.choice(elig)
            pos = random.choice(positions)
            entries.append((eid, pid, pos, sd, None))
            eid += 1

        for sd in start_dates[remaining_current:]:
            cutoff = sd - timedelta(days=365 * min_hire_age)
            elig = [pid for pid, dob in persons if dob <= cutoff]
            if not elig:
                continue
            pid = random.choice(elig)
            pos = random.choice(positions)
            max_end = min(today, sd + timedelta(days=365 * 3))
            delta_days = (max_end - sd).days
            if delta_days >= 30:
                ed = sd + timedelta(days=random.randint(30, delta_days))
            else:
                # Jeśli nie ma wystarczającego zakresu, ustaw losowo np. 30 dni lub max_end
                ed = max_end

            entries.append((eid, pid, pos, sd, ed))
            eid += 1

        cur.executemany(
            "INSERT INTO PRACOWNICY (ID_PRACOWNIKA, ID_OSOBY, STANOWISKO, DATA_POCZATEK, DATA_KONIEC) VALUES (%s,%s,%s,%s,%s)",
            entries
        )
        conn.commit()
        print(f"Wstawiono {len(entries)} pracowników, w tym {len(positions)} na dzień startu.")

    except:
        conn.rollback()
        raise

    finally:
        cur.close()
        conn.close()

if __name__ == '__main__':
    fill_PRACOWNICY()

Wstawiono 80 pracowników, w tym 4 na dzień startu.


## Płatności, podróże, obsługa, pasażerowie

In [ ]:
failure_rate = 0.005
ongoing_rate = 0.05
trips_per_employee = 0.25
price_per_km = 9.0

# funkcja do kalkulowania strat w przypadku nieudanej podróży
def compute_loss(C0, D, R_max, M, R_lot, p_km):
    A = min(1.0, D / R_max)
    loss_rocket = (1 - A) * C0 / 10
    loss_passengers = M * p_km * R_lot
    return loss_rocket + loss_passengers

def fill_travel_data():
    conn = mysql.connector.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    current_datetime = datetime.now()

    try:
        cur.execute('SET FOREIGN_KEY_CHECKS=0;')
        for tbl in ['OBSLUGA','PASAZEROWIE','PODROZE','PLATNOSCI']:
            cur.execute(f'TRUNCATE TABLE {tbl};')
        cur.execute('SET FOREIGN_KEY_CHECKS=1;')

        cur.execute('SELECT ID_TRASY, DLUGOSC_TRASY, ID_STACJA_START FROM TRASY;')
        routes = cur.fetchall()

        cur.execute('SELECT ID_POJAZDU, PREDKOSC_MAX, ZASIEG_KM, POJEMNOSC FROM POJAZDY;')
        vehicles = cur.fetchall()

        cur.execute('SELECT ID_PRACOWNIKA, DATA_POCZATEK, DATA_KONIEC FROM PRACOWNICY;')
        employees = cur.fetchall()

        # mapowanie ID_OSOBY -> ID_STACJI
        cur.execute('SELECT ID_OSOBY, ID_STACJI FROM OSOBY;')
        person_to_station = {pid: sid for pid, sid in cur.fetchall()}
        persons = list(person_to_station.keys())

        # mapowanie ID_STACJI -> ID_OSOBY
        station_pax = {}
        for pid, sid in person_to_station.items():
            station_pax.setdefault(sid, []).append(pid)

        # Okres 3 lat wstecz od dzisiaj
        inception_date = current_datetime.date() - timedelta(days=365 * years_active)
        year_bounds = []
        for i in range(years_active):
            start = inception_date + timedelta(days=365 * i)
            end = inception_date + timedelta(days=365 * (i + 1))
            if end > current_datetime.date():
                end = current_datetime.date()
            year_bounds.append((start, end))

        payments, trips, crew_list, pax_list = [], [], [], []
        pay_id = 1
        trip_id = 1

        # historia podróży dla osób
        pax_trips = {pid: [] for pid in persons}

        for ys, ye in year_bounds:
            # Pracownicy aktywni w danym roku
            active = [e for e in employees if e[1] <= ye and (e[2] is None or e[2] >= ys)]
            num_trips = int(len(active) * trips_per_employee)
            
            for _ in range(num_trips):
                route_id, dist, start_st = random.choice(routes)
                
                # generowanie daty z losową godziną
                start_date = ys + timedelta(days=random.randint(0, (ye - ys).days))
                hour = random.randint(6, 22)
                minute = random.randint(0, 59)
                second = random.randint(0, 59)
                sd = datetime(start_date.year, start_date.month, start_date.day, hour, minute, second)

                # data zakończenia nie może być w przyszłości
                if sd > current_datetime:
                    sd = current_datetime - timedelta(
                        days=random.randint(1, 30),
                        hours=random.randint(0, 23),
                        minutes=random.randint(0, 59)
                    )

                vid, speed_max, r_max, capacity = random.choice(vehicles)
                hours = dist / speed_max * random.uniform(0.9, 1.1)
                duration = timedelta(hours=hours)
                ed = None

                r = random.random()
                if r < failure_rate:
                    status = 'nieudana'
                    ed = None
                elif r < failure_rate + ongoing_rate:
                    status = 'w trakcie'
                    sd = current_datetime - timedelta(
                        days=random.randint(0, 30),
                        hours=random.randint(0, 23),
                        minutes=random.randint(0, 59))
                else:
                    status = 'zakonczona'
                    ed = sd + duration
                    
                    # jeśli kończy się w przyszłości to przesuwamy start wstecz
                    if ed > current_datetime:
                        time_over = ed - current_datetime
                        sd = sd - time_over - timedelta(hours=random.randint(1, 24))
                        ed = sd + duration
                        
                        # skróć czas lotu
                        if ed > current_datetime:
                            max_duration = current_datetime - sd
                            hours = min(hours, max_duration.total_seconds() / 3600)
                            duration = timedelta(hours=hours)
                            ed = sd + duration

                station_pax_list = station_pax.get(start_st, [])
                if not station_pax_list:
                    continue

                # losowanie liczby pasażerów (min:5, max:capacity)
                num_pax = min(len(station_pax_list), random.randint(5, capacity))
                if num_pax == 0:
                    continue

                selected_pax = []
                attempts = 0
                while len(selected_pax) < num_pax and attempts < 100:
                    pid = random.choice(station_pax_list)
                    attempts += 1

                    collision = False
                    current_trip_end = ed if ed is not None else current_datetime

                    for p_start, p_end in pax_trips.get(pid, []):
                        if p_end is None:
                            p_end = current_datetime

                        if sd <= p_end and current_trip_end >= p_start:
                            collision = True
                            break

                    if not collision:
                        selected_pax.append(pid)
                        pax_trips[pid].append((sd, current_trip_end))

                if len(selected_pax) == 0:
                    continue

                base_cost = dist * price_per_km / 1000
                cost_completed = round(base_cost * (1 + len(selected_pax) * 0.05), 2)

                fill_threshold = math.ceil(0.5 * capacity)
                ticket_price = round((cost_completed / fill_threshold) if fill_threshold > 0 else 0, 2)
                revenue = round(ticket_price * len(selected_pax), 2)
                
                cost = None
                if status == 'nieudana':
                    M = random.randint(1, min(capacity, len(selected_pax)))
                    C0 = base_cost * 1000
                    cost = round(compute_loss(C0, dist, r_max, M, dist, price_per_km), 2)
                elif status == 'zakonczona':
                    cost = cost_completed

                methods = ['karta', 'gotowka']
                method = random.choice(methods)

                payments.append((pay_id, revenue, 'PLN', sd.date(), method))
                trips.append((
                    trip_id, route_id, vid,
                    sd,
                    ed,
                    cost, pay_id, status
                ))

                crew = random.sample(active, min(2, len(active))) if active else []
                for emp in crew:
                    crew_list.append((trip_id, emp[0]))

                for pid in selected_pax:
                    aboard = 1 if status == 'zakonczona' else 0
                    pax_list.append((trip_id, pid, pay_id, aboard))

                trip_id += 1
                pay_id += 1

        cur.executemany('INSERT INTO PLATNOSCI VALUES (%s,%s,%s,%s,%s)', payments)
        cur.executemany('INSERT INTO PODROZE VALUES (%s,%s,%s,%s,%s,%s,%s,%s)', trips)
        cur.executemany('INSERT INTO OBSLUGA VALUES (%s,%s)', crew_list)
        cur.executemany('INSERT INTO PASAZEROWIE VALUES (%s,%s,%s,%s)', pax_list)
        conn.commit()
        print('Uzupełniono tabele z właściwą logiką kosztów i przychodów')

    except Exception as e:
        conn.rollback()
        print(f"Błąd: {e}")
        raise e
    finally:
        cur.close()
        conn.close()

if __name__ == '__main__':
    fill_travel_data()

Uzupełniono tabele z właściwą logiką kosztów i przychodów


## Garaż, serwis

In [ ]:
GARAGE_COUNTS = {
            1: 5,
            2: 5,
            3: 5,
            4: 5,
            5: 5,
            6: 7,
            7: 5,
            8: 5,
            9: 5,
            10: 5
        }

SERVICE_TYPES = [
    ('naprawa powłoki', 8000, 20000),
    ('naprawa systemu hamulcowego', 10000, 30000),
    ('serwis silnika', 20000, 100000),
    ('wymiana akumulatora', 15000, 50000),
    ('diagnoza elektroniczna', 5000, 20000),
    ('naprawa układu napędowego', 30000, 150000),
    ('konserwacja ogólna', 5000, 40000)
]


def fill_GARAZ_SERWIS():
    conn = mysql.connector.connect(**DB_CONFIG)
    cur = conn.cursor()
    try:
        cur.execute('SET FOREIGN_KEY_CHECKS=0;')
        cur.execute('TRUNCATE TABLE SERWIS;')
        cur.execute('TRUNCATE TABLE GARAZ;')
        cur.execute('SET FOREIGN_KEY_CHECKS=1;')
        conn.commit()

        cur.execute('SELECT ID_POJAZDU FROM POJAZDY;')
        vehicles = [row[0] for row in cur.fetchall()]

        garaz_entries = []
        machine_id = 1
        for v in vehicles:
            count = GARAGE_COUNTS.get(v, 1)
            for _ in range(count):
                status = random.choice(['dostepny', 'serwis'])
                garaz_entries.append((machine_id, v, status))
                machine_id += 1

        cur.executemany(
            'INSERT INTO GARAZ (ID_MASZYNY, ID_POJAZDU, STATUS) VALUES (%s, %s, %s)',
            garaz_entries
        )
        conn.commit()

        cur.execute('SELECT ID_MASZYNY FROM GARAZ;')
        machines = [row[0] for row in cur.fetchall()]

        serv_entries = []
        service_id = 1
        inception = datetime.now().date() - timedelta(days=365 * years_active)
        today = datetime.now().date()

        for m in machines:
            num_events = random.randint(years_active, years_active * 3)
            last_end = inception
            for _ in range(num_events):
                start = last_end + timedelta(days=random.randint(0, 365))
                if start > today - timedelta(days=1):
                    break
                end = start + timedelta(days=random.randint(1,14))
                if end > today:
                    end = today
                service_name, min_cost, max_cost = random.choice(SERVICE_TYPES)
                cost = round(random.uniform(min_cost, max_cost), 2)
                description = service_name
                serv_entries.append((service_id, m, start, end, description, cost))
                service_id += 1
                last_end = end + timedelta(days=random.randint(0, 30))
                if last_end > today:
                    break

        cur.executemany(
            'INSERT INTO SERWIS (ID_SERWISU, ID_MASZYNY, DATA_ROZPOCZECIA, DATA_ZAKONCZENIA, OPIS, KOSZT) VALUES (%s, %s, %s, %s, %s, %s)',
            serv_entries
        )
        conn.commit()

        cur.execute("SELECT DISTINCT ID_POJAZDU FROM PODROZE WHERE STATUS='w trakcie';")
        ongoing_models = [r[0] for r in cur.fetchall()]
        for model_id in ongoing_models:
            cur.execute('SELECT ID_MASZYNY FROM GARAZ WHERE ID_POJAZDU=%s LIMIT 1;', (model_id,))
            row = cur.fetchone()
            if row:
                machine = row[0]
                cur.execute('UPDATE GARAZ SET STATUS=%s WHERE ID_MASZYNY=%s;', ('na misji', machine))
        conn.commit()

        print(f"Inserted {len(garaz_entries)} GARAZ entries and {len(serv_entries)} SERWIS events, updated ongoing missions.")

    except Exception:
        conn.rollback()
        raise
    finally:
        cur.close()
        conn.close()

if __name__ == '__main__':
    fill_GARAZ_SERWIS()

Inserted 52 GARAZ entries and 390 SERWIS events, updated ongoing missions.
